In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
json_data = 'cropped_data/cropped_annotation/cropped_annotation_mark_added.json'

json_object = json.load(open(json_data, "r"))
#json_formatted_str = json.dumps(json_object, indent=2)

## Step of preprocessing data

1. Count lesions, marking, pos, neg of each chunk of data (train/val/test)
2. Crop image according to 'window' create cropped image folder 
3. Create new dataset of cropped dataset (create new json file with new bounding box)

#### JSON USG cheat

dataset type => d['set']

lesions => d['regions'][i][‘region_attributes’][‘structure’]

boundingg box => d[‘regions’][i][‘bbox’]

##################################################################

count dataset [FFC, FFS, hemangioma, HCC, cyst]
count marking 
count 
separate into training, vilidation, testing

In [ ]:
interest_classes = ['FFC', 'FFS', 'hemangioma', 'HCC', 'cyst']

def count_in_dataset(dataset_type):
    dataset = dict()
    
    for k,v in json_object.items():
        # Initialize the variable
        pos = False
        
        # Get dataset type
        data_type = v['set']
        
        # Check consistent of image and dataset
        if data_type.split("_")[0].strip() == dataset_type:
            # Find lesion in the image
            regions = v['regions']
            
            found_mark = False
            # Loop through the regions
            for l in regions:

                if 'structure' in l['region_attributes'].keys():
                    if l['region_attributes']['structure'] in interest_classes:
                        pos = True
                        # Add structure into the dataset type
                        struc = l['region_attributes']['structure']
                        if struc not in dataset.keys():
                            dataset[struc] = 1
                        else:
                            dataset[struc] += 1

                elif 'markings' in l['region_attributes'].keys() and not found_mark:
                    # Add marking into the dataset type
                    if 'mark' not in dataset.keys():
                        dataset['mark'] = 1
                    else:
                        dataset['mark'] += 1 
                    found_mark = True
            if pos:
                if 'pos' not in dataset.keys():
                        dataset['pos'] = 1
                else:
                    dataset['pos'] += 1
            else:
                if 'neg' not in dataset.keys():
                        dataset['neg'] = 1
                else:
                    dataset['neg'] += 1
        
    return dataset

In [ ]:
r = 0
for k,v in json_object.items():
    print(v)
    r += 1
    if r > 1000:
        break

In [ ]:
# Each type of dataset should have following key => FFC, FFS, hemangioma, HCC, cyst, marking, pos, neg

training_des = count_in_dataset('train')
validation_des = count_in_dataset('val')
testing_des = count_in_dataset('test')

In [ ]:
training_des

In [ ]:
validation_des

In [ ]:
testing_des

In [ ]:
sorted(training_des.items())

In [ ]:
# create list of values in each dataset
train_list = [v for k, v in sorted(training_des.items())]
val_list = [v for k, v in sorted(validation_des.items())]
test_list = [v for k, v in sorted(testing_des.items())]

In [ ]:
print("Lesion in training set:",sum(train_selected)) # 0.878
print("Lesion in validation set:",sum(val_selected)) # 0.0586
print("Lesion in testing set:",sum(test_selected)) #  0.0634
print("Al lesion in dataset", sum(train_selected)+sum(val_selected)+sum(test_selected))

In [ ]:
train_selected = train_list[:-3]
train_ratio = [x / sum(train_selected) for x in train_selected ]
print(train_ratio)

In [ ]:
val_selected = val_list[:-3]
val_ratio = [x / sum(val_selected) for x in val_selected ]
print(val_ratio)

In [ ]:
test_selected = test_list[:-3]
test_ratio = [x / sum(test_selected) for x in test_selected ]
print(test_ratio)

In [ ]:
labels = ['FFC', 'FFS', 'HCC', 'cyst', 'hemangioma', 'mark', 'neg', 'pos']
x = np.arange(len(labels))
width = 0.25

fig, ax = plt.subplots(figsize=(12,8))
rects1 = ax.bar(x - width, train_list, width, label='Train')
rects2 = ax.bar(x, val_list, width, label='Val')
rects3 = ax.bar(x + width, test_list, width, label='Test')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('The number of each region')
ax.set_title('Each of region in the dataset')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

In [ ]:
print(train_list[6:],sum(train_list[6:])) #0.7865, 0.2134
print(val_list[6:], sum(val_list[6:])) # 0.8577, 
print(test_list[6:], sum(test_list[6:])) # 0.8572

# All images = 99541
# Training images => 81929 / 99541 = 0.82
# Validation images => 8624 / 99541 = 0.0866
# Testing images => 8988 / 99541 = 0.09